## Hello custom training: Training a custom image classification model

Reference: https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom

In [1]:
import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds

IMG_WIDTH = 128

## User Config

In [5]:
gcp_project = 'zproject201807'
gcp_region  = 'us-central1'

## Training Code

In [6]:
# Set environ variables

os.environ['GCP_PROJECT']=gcp_project

gcp_bucket_path = '{}-ai'.format(gcp_project)

os.environ['AIP_MODEL_DIR']='{}/output'.format(gcp_bucket_path)

In [7]:
def normalize_img(image):
    """Normalizes image.
    
    * Resizes image to IMG_WIDTH x IMG_WIDTH pixels
    * Casts values from `uint8` to `float32`
    * Scales values from [0, 255] to [0, 1]
    
    Returns:
      A tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color channels)
    """
    image = tf.image.resize_with_pad(image, IMG_WIDTH, IMG_WIDTH)
    return image / 255.

In [8]:
def normalize_img_and_label(image, label):
    """Normalizes image and label.
    
    * Performs normalize_img on image
    * Passes through label unchanged
    
    Returns:
      Tuple (image, label) where
      * image is a tensor with shape (IMG_WIDTH, IMG_WIDTH, 3). (3 color
        channels)
      * label is an unchanged integer [0, 4] representing flower type
    """
    return normalize_img(image), label


In [9]:
if 'AIP_MODEL_DIR' not in os.environ:
    raise KeyError(
        'The `AIP_MODEL_DIR` environment variable has not been' +
        'set. See https://cloud.google.com/ai-platform-unified/docs/tutorials/image-recognition-custom/training'
    )

output_directory = os.environ['AIP_MODEL_DIR']
output_directory = os.environ['AIP_MODEL_DIR']

In [12]:
dataset = tfds.load('tf_flowers:3.*.*',
                    split='train',
                    try_gcs=True,
                    shuffle_files=True,
                    as_supervised=True)

In [13]:
dataset = dataset.map(normalize_img_and_label,
    num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [14]:
dataset = dataset.cache()
dataset = dataset.shuffle(1000)
dataset = dataset.batch(128)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,
                           3,
                           padding='same',
                           activation='relu',
                           input_shape=(IMG_WIDTH, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(5)  # 5 classes
])

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [17]:
model.fit(dataset, epochs=10)

Epoch 1/10
29/29 [==============================] - 25s 855ms/step - loss: 1.3816 - accuracy: 0.4084
Epoch 2/10
29/29 [==============================] - 22s 743ms/step - loss: 1.0782 - accuracy: 0.5678
Epoch 3/10
29/29 [==============================] - 22s 770ms/step - loss: 0.9334 - accuracy: 0.6406
Epoch 4/10
29/29 [==============================] - 22s 748ms/step - loss: 0.8290 - accuracy: 0.6820
Epoch 5/10
29/29 [==============================] - 22s 748ms/step - loss: 0.7351 - accuracy: 0.7270
Epoch 6/10
29/29 [==============================] - 22s 745ms/step - loss: 0.6382 - accuracy: 0.7613
Epoch 7/10
29/29 [==============================] - 22s 752ms/step - loss: 0.5451 - accuracy: 0.7929
Epoch 8/10
29/29 [==============================] - 22s 765ms/step - loss: 0.4490 - accuracy: 0.8420
Epoch 9/10
29/29 [==============================] - 22s 758ms/step - loss: 0.3974 - accuracy: 0.8529
Epoch 10/10
29/29 [==============================] - 21s 741ms/step - loss: 0.2950 - accura

In [18]:
print(f'Exporting SavedModel to: {output_directory}')

Exporting SavedModel to: zproject201807-ai/output


In [20]:
# Add softmax layer for intepretability
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [21]:
probability_model.save(output_directory)

INFO:tensorflow:Assets written to: zproject201807-ai/output/assets


INFO:tensorflow:Assets written to: zproject201807-ai/output/assets
